# The Ames (Iowa) housing dataset

Description of the data:
- 1460 samples, 79 features
- contains both numerical and categorical data


In [4]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from IPython.display import display
from pandas.api.types import CategoricalDtype

from category_encoders import MEstimateEncoder
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.feature_selection import mutual_info_regression
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import mean_absolute_error
from xgboost import XGBRegressor

# Preprocessing the data

In [5]:
def load_data():
    
    # Read data
    df_train = pd.read_csv("./data/train.csv", index_col="Id")
    df_test = pd.read_csv("./data/test.csv", index_col="Id")
    
    # Merge the splits so we can process them together
    df = pd.concat([df_train, df_test])
    
    # Preprocessing
    df = clean(df)
    df = encode(df)
    df = impute(df)
    
    # Reform splits
    df_train = df.loc[df_train.index, :]
    df_test = df.loc[df_test.index, :]
    return df_train, df_test

## Data cleaning

In [6]:
# Defining the cleaning function

def clean(df):
    df["Exterior2nd"] = df["Exterior2nd"].replace({"Brk Cmn": "BrkComm"})
    
    # Replacing corrupt values in GarageYrBlt with the year the house was built
    df["GarageYrBlt"] = df["GarageYrBlt"].where(df.GarageYrBlt <= 2010, df.YearBuilt)
    
    # Renaming column names that begin with a number just in case they cause problems
    df.rename(columns={
        "1stFlrSF": "FirstFlrSF",
        "2ndFlrSF": "SecondFlrSF",
        "3SsnPorch": "Threeseasonporch",
    }, inplace=True,
    )
    return df

## Feature encoding
Encoding each feature with its correct type helps ensure each feature is treated appropriately by whatever functions we use, and makes it easier for us to apply transformations consistently.

The numeric features are already encoded correctly ('float' for continuous, 'int' for discrete), but the categoricals we'll need to do ourselves. Note in particular, that the 'MSSubClass' feature is read as an 'int' type, but is actually a (nominative) categorical.

In [7]:
# Defining the encoding function

# The nominative (unordered) categorical features
features_nom = ["MSSubClass", "MSZoning", "Street", "Alley", "LandContour", "LotConfig", "Neighborhood", "Condition1", "Condition2", "BldgType", "HouseStyle", "RoofStyle", "RoofMatl", "Exterior1st", "Exterior2nd", "MasVnrType", "Foundation", "Heating", "CentralAir", "GarageType", "MiscFeature", "SaleType", "SaleCondition"]

# The ordinal (ordered) categorical features 
# Pandas calls the categories "levels"
five_levels = ["Po", "Fa", "TA", "Gd", "Ex"]
ten_levels = list(range(10))

ordered_levels = {
    "OverallQual": ten_levels,
    "OverallCond": ten_levels,
    "ExterQual": five_levels,
    "ExterCond": five_levels,
    "BsmtQual": five_levels,
    "BsmtCond": five_levels,
    "HeatingQC": five_levels,
    "KitchenQual": five_levels,
    "FireplaceQu": five_levels,
    "GarageQual": five_levels,
    "GarageCond": five_levels,
    "PoolQC": five_levels,
    "LotShape": ["Reg", "IR1", "IR2", "IR3"],
    "LandSlope": ["Sev", "Mod", "Gtl"],
    "BsmtExposure": ["No", "Mn", "Av", "Gd"],
    "BsmtFinType1": ["Unf", "LwQ", "Rec", "BLQ", "ALQ", "GLQ"],
    "BsmtFinType2": ["Unf", "LwQ", "Rec", "BLQ", "ALQ", "GLQ"],
    "Functional": ["Sal", "Sev", "Maj1", "Maj2", "Mod", "Min2", "Min1", "Typ"],
    "GarageFinish": ["Unf", "RFn", "Fin"],
    "PavedDrive": ["N", "P", "Y"],
    "Utilities": ["NoSeWa", "NoSewr", "AllPub"],
    "CentralAir": ["N", "Y"],
    "Electrical": ["Mix", "FuseP", "FuseF", "FuseA", "SBrkr"],
    "Fence": ["MnWw", "GdWo", "MnPrv", "GdPrv"],
}

# Add a None level for missing values
ordered_levels = {key: ["None"] + value for key, value in
                  ordered_levels.items()}

def encode(df):
    # Nominal categories
    for name in features_nom:
        df[name] = df[name].astype("category")
        
        # Add a None category for missing values
        if "None" not in df[name].cat.categories:
            df[name] = df[name].cat.add_categories("None")
    
    # Ordinal categories
    for name, levels in ordered_levels.items():
        df[name] = df[name].astype(CategoricalDtype(levels,
                                                    ordered=True))
    return df

## Dealing with missing values

We'll impute 0 for missing numeric values and "None" for missing categorical values.

In [8]:
def impute(df):
    for name in df.select_dtypes("number"):
        df[name] = df[name].fillna(0)
    for name in df.select_dtypes("category"):
        df[name] = df[name].fillna("None")
    return df

## Establishing a baseline score

In [9]:
# Loading the data
df_train, df_test = load_data()

In [10]:
# Looking at the values
display(df_train)
display(df_test)

# Displaying information about dtypes and missing values
display(df_train.info())
display(df_test.info())

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,
1,60,RL,65.0,8450,Pave,None,Reg,Lvl,AllPub,Inside,...,0,None,None,None,0,2,2008,WD,Normal,208500.0
2,20,RL,80.0,9600,Pave,None,Reg,Lvl,AllPub,FR2,...,0,None,None,None,0,5,2007,WD,Normal,181500.0
3,60,RL,68.0,11250,Pave,None,IR1,Lvl,AllPub,Inside,...,0,None,None,None,0,9,2008,WD,Normal,223500.0
4,70,RL,60.0,9550,Pave,None,IR1,Lvl,AllPub,Corner,...,0,None,None,None,0,2,2006,WD,Abnorml,140000.0
5,60,RL,84.0,14260,Pave,None,IR1,Lvl,AllPub,FR2,...,0,None,None,None,0,12,2008,WD,Normal,250000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1456,60,RL,62.0,7917,Pave,None,Reg,Lvl,AllPub,Inside,...,0,None,None,None,0,8,2007,WD,Normal,175000.0
1457,20,RL,85.0,13175,Pave,None,Reg,Lvl,AllPub,Inside,...,0,None,MnPrv,None,0,2,2010,WD,Normal,210000.0
1458,70,RL,66.0,9042,Pave,None,Reg,Lvl,AllPub,Inside,...,0,None,GdPrv,Shed,2500,5,2010,WD,Normal,266500.0


,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,
1461,20,RH,80.0,11622,Pave,None,Reg,Lvl,AllPub,Inside,...,0,None,MnPrv,None,0,6,2010,WD,Normal,0.0
1462,20,RL,81.0,14267,Pave,None,IR1,Lvl,AllPub,Corner,...,0,None,None,Gar2,12500,6,2010,WD,Normal,0.0
1463,60,RL,74.0,13830,Pave,None,IR1,Lvl,AllPub,Inside,...,0,None,MnPrv,None,0,3,2010,WD,Normal,0.0
1464,60,RL,78.0,9978,Pave,None,IR1,Lvl,AllPub,Inside,...,0,None,None,None,0,6,2010,WD,Normal,0.0
1465,120,RL,43.0,5005,Pave,None,IR1,HLS,AllPub,Inside,...,0,None,None,None,0,1,2010,WD,Normal,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2915,160,RM,21.0,1936,Pave,None,Reg,Lvl,AllPub,Inside,...,0,None,None,None,0,6,2006,WD,Normal,0.0
2916,160,RM,21.0,1894,Pave,None,Reg,Lvl,AllPub,Inside,...,0,None,None,None,0,4,2006,WD,Abnorml,0.0
2917,20,RL,160.0,20000,Pave,None,Reg,Lvl,AllPub,Inside,...,0,None,None,None,0,9,2006,WD,Abnorml,0.0


<class 'pandas.core.frame.DataFrame'>
Index: 1460 entries, 1 to 1460
Data columns (total 80 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   MSSubClass        1460 non-null   category
 1   MSZoning          1460 non-null   category
 2   LotFrontage       1460 non-null   float64 
 3   LotArea           1460 non-null   int64   
 4   Street            1460 non-null   category
 5   Alley             1460 non-null   category
 6   LotShape          1460 non-null   category
 7   LandContour       1460 non-null   category
 8   Utilities         1460 non-null   category
 9   LotConfig         1460 non-null   category
 10  LandSlope         1460 non-null   category
 11  Neighborhood      1460 non-null   category
 12  Condition1        1460 non-null   category
 13  Condition2        1460 non-null   category
 14  BldgType          1460 non-null   category
 15  HouseStyle        1460 non-null   category
 16  OverallQual       1460 non-nu

None

<class 'pandas.core.frame.DataFrame'>
Index: 1459 entries, 1461 to 2919
Data columns (total 80 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   MSSubClass        1459 non-null   category
 1   MSZoning          1459 non-null   category
 2   LotFrontage       1459 non-null   float64 
 3   LotArea           1459 non-null   int64   
 4   Street            1459 non-null   category
 5   Alley             1459 non-null   category
 6   LotShape          1459 non-null   category
 7   LandContour       1459 non-null   category
 8   Utilities         1459 non-null   category
 9   LotConfig         1459 non-null   category
 10  LandSlope         1459 non-null   category
 11  Neighborhood      1459 non-null   category
 12  Condition1        1459 non-null   category
 13  Condition2        1459 non-null   category
 14  BldgType          1459 non-null   category
 15  HouseStyle        1459 non-null   category
 16  OverallQual       1459 non

None

## Notes on using MAE

Mean Absolute Error (MAE) measures the average size of the mistakes in a collection of predictions, without taking their direction into account. It takes the average absolute difference between the predicted values and the actual values.

In [11]:
# Using XGB to compute the cross-validated MAE score for our feature set

def score_dataset_XGB(X, y, model=XGBRegressor()):
    # Label encoding for categoricals
    for colname in X.select_dtypes(["category"]):
        X[colname] = X[colname].cat.codes
    
    # Using MAE (mean absolute error)
    score = cross_val_score(
        model, X, y, cv=5, scoring="neg_mean_squared_error",
    )
    score = -1 * score.mean()
    score = np.sqrt(score)
    return score

In [12]:
# using Random Forest for scoring

from sklearn.ensemble import RandomForestRegressor

def score_dataset_RF(X, y, model=RandomForestRegressor()):
    # Label encoding for categoricals
    for colname in X.select_dtypes(["category"]):
        X[colname] = X[colname].cat.codes
    
   # Using MAE (mean absolute error)
    score = cross_val_score(
        model, X, y, cv=5, scoring="neg_mean_squared_error",
    )
    score = -1 * score.mean()
    score = np.sqrt(score)
    return score

In [13]:
X = df_train.copy()
y = X.pop("SalePrice")

baseline_score_xgb = score_dataset_XGB(X, y)
print(f"Baseline score: {baseline_score_xgb:.5f} MAE")

Baseline score: 30839.49835 MAE


In [14]:
baseline_score_rf = score_dataset_RF(X, y)
print(f"Baseline score: {baseline_score_rf:.5f} MAE")

Baseline score: 28938.75569 MAE


## Determining feature utility

Usually, we want to focus on the top scoring features the most during feature development. Training on uninformative features can lead to overfitting.

In [19]:
# using permutation importance

from sklearn.inspection import permutation_importance

X = df_train.copy()
y = X.pop("SalePrice")

for colname in X.select_dtypes(["category"]):
        X[colname] = X[colname].cat.codes

model = RandomForestRegressor()
model.fit(X, y)

# performing permutation importance
importance = permutation_importance(model, X, y, n_repeats=10, n_jobs=2, random_state=123)

# sorting features by average decrease in accuracy
sorted_idx = importance.importances_mean.argsort()

# organizing data
importance_dict = {
    'Feature': X.columns.values[sorted_idx],
    'Importance': importance.importances_mean[sorted_idx]
}

# creating and sorting a dataframe
importance_df = pd.DataFrame(importance_dict).sort_values('Importance', ascending=False)

In [79]:
importance_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 79 entries, 78 to 0
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Feature     79 non-null     object 
 1   Importance  79 non-null     float64
dtypes: float64(1), object(1)
memory usage: 3.9+ KB


In [21]:
importance_df.tail(n=15)

,Feature,Importance
14,GarageCond,1.374486e-04
13,RoofMatl,1.284815e-04
12,BsmtFinType2,1.277304e-04
11,PoolArea,1.102087e-04
10,LowQualFinSF,7.753901e-05
9,Electrical,6.951168e-05
8,BsmtHalfBath,6.569984e-05
7,Threeseasonporch,6.263436e-05
6,Heating,6.160252e-05
5,Alley,4.320813e-05


In [43]:
# using mutual information

def make_mi_scores(X, y):
    X = X.copy()
    for colname in X.select_dtypes(["object", "category"]):
        X[colname], _ = X[colname].factorize()
    # All discrete features should now have integer dtypes
    
    discrete_features = [pd.api.types.is_integer_dtype(t) for t in X.dtypes]
    mi_scores = mutual_info_regression(X, y, discrete_features=discrete_features, random_state=0)
    mi_scores = pd.Series(mi_scores, name="MI Scores", index=X.columns)
    mi_scores = mi_scores.sort_values(ascending=False)
    return mi_scores

In [44]:
X = df_train.copy()
y = X.pop("SalePrice")

mi_scores = make_mi_scores(X, y)
mi_scores.head(n=15)

OverallQual     0.571457
Neighborhood    0.526220
GrLivArea       0.430395
YearBuilt       0.407974
LotArea         0.394468
TotalBsmtSF     0.368536
GarageArea      0.361542
GarageCars      0.352312
BsmtQual        0.330803
KitchenQual     0.324679
ExterQual       0.324003
FirstFlrSF      0.282503
MSSubClass      0.281349
YearRemodAdd    0.274032
GarageYrBlt     0.272213
Name: MI Scores, dtype: float64

In [84]:
importance = importance_df[importance_df['Importance'] > 0.001]
importance.head()

,Feature,Importance
78,GrLivArea,0.198671
77,ExterQual,0.107631
76,GarageCars,0.078120
75,OverallQual,0.061650
74,TotalBsmtSF,0.038520


In [85]:
X = df_train.copy()
y = X.pop("SalePrice")

X.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
Id,,,,,,,,,,,,,,,,,,,,,
1,60,RL,65.0,8450,Pave,None,Reg,Lvl,AllPub,Inside,...,0,0,None,None,None,0,2,2008,WD,Normal
2,20,RL,80.0,9600,Pave,None,Reg,Lvl,AllPub,FR2,...,0,0,None,None,None,0,5,2007,WD,Normal
3,60,RL,68.0,11250,Pave,None,IR1,Lvl,AllPub,Inside,...,0,0,None,None,None,0,9,2008,WD,Normal
4,70,RL,60.0,9550,Pave,None,IR1,Lvl,AllPub,Corner,...,0,0,None,None,None,0,2,2006,WD,Abnorml
5,60,RL,84.0,14260,Pave,None,IR1,Lvl,AllPub,FR2,...,0,0,None,None,None,0,12,2008,WD,Normal


In [91]:
importance_s = importance['Importance']
importance_s.index = importance_df['Feature']
importance_s = importance_s.sort_values(ascending=False)
importance_s.head()

ValueError: Length mismatch: Expected axis has 35 elements, new values have 79 elements

In [15]:
# def drop_uninformative(df, mi_scores):
    #return df.loc[:, mi_scores > 0.0]

In [45]:
print(type(mi_scores))

<class 'pandas.core.series.Series'>


IndexingError: Unalignable boolean Series provided as indexer (index of the boolean Series and of the indexed object do not match).

In [16]:


mi_score_xgb = score_dataset_XGB(X, y)
print(f"MI score: {mi_score_xgb:.5f} MAE")

mi_score_rf = score_dataset_RF(X, y)
print(f"MI score: {mi_score_rf:.5f} MAE")

MI score: 30463.02667 MAE
MI score: 28832.88444 MAE


## Creating the features

In [17]:
#  using label encoding for the categorical features

def label_encode(df):
    X = df.copy()
    for colname in X.select_dtypes(["category"]):
        X[colname] = X[colname].cat.codes
    return X

In [18]:
def create_features(df, df_test=None):
    X = df.copy()
    y = X.pop("SalePrice")
    mi_scores = make_mi_scores(X, y)

    # Combine splits if test data is given
    #
    # If we're creating features for test set predictions, we should
    # use all the data we have available. After creating our features,
    # we'll recreate the splits.
    if df_test is not None:
        X_test = df_test.copy()
        X_test.pop("SalePrice")
        X = pd.concat([X, X_test])

    # Mutual Information
    X = drop_uninformative(X, mi_scores)

    X = label_encode(X)

    # Reform splits
    if df_test is not None:
        X_test = X.loc[df_test.index, :]
        X.drop(df_test.index, inplace=True)
    
    if df_test is not None:
        return X, X_test
    else:
        return X

In [19]:
df_train, df_test = load_data()
X_train = create_features(df_train)
y_train = df_train.loc[:, "SalePrice"]

score_dataset_XGB(X_train, y_train)

30463.026666098118

In [20]:
score_dataset_RF(X_train, y_train)

29556.468711140835

## Tuning the model's hyperparameters

In [21]:
X_train = create_features(df_train)
y_train = df_train.loc[:, "SalePrice"]

xgb_params = dict(
    max_depth=6,           # maximum depth of each tree - try 2 to 10
    learning_rate=0.01,    # effect of each tree - try 0.0001 to 0.1
    n_estimators=1000,     # number of trees (that is, boosting rounds) - try 1000 to 8000
    min_child_weight=1,    # minimum number of houses in a leaf - try 1 to 10
    colsample_bytree=0.7,  # fraction of features (columns) per tree - try 0.2 to 1.0
    subsample=0.7,         # fraction of instances (rows) per tree - try 0.2 to 1.0
    reg_alpha=0.5,         # L1 regularization (like LASSO) - try 0.0 to 10.0
    reg_lambda=1.0,        # L2 regularization (like Ridge) - try 0.0 to 10.0
    num_parallel_tree=1,   # set > 1 for boosted random forests
)

In [23]:
xgb = XGBRegressor(**xgb_params)
score_dataset_XGB(X_train, y_train, xgb)

26846.0119649431

### Trying out an automatic hyperparameter tuner

In [24]:
pip install optuna

Note: you may need to restart the kernel to use updated packages.


In [25]:
import optuna

def objective(trial):
    xgb_params_opt = dict(
        max_depth=trial.suggest_int("max_depth", 2, 10),
        learning_rate=trial.suggest_float("learning_rate", 1e-4, 1e-1, log=True),
        n_estimators=trial.suggest_int("n_estimators", 1000, 8000),
        min_child_weight=trial.suggest_int("min_child_weight", 1, 10),
        colsample_bytree=trial.suggest_float("colsample_bytree", 0.2, 1.0),
        subsample=trial.suggest_float("subsample", 0.2, 1.0),
        reg_alpha=trial.suggest_float("reg_alpha", 1e-4, 1e2, log=True),
        reg_lambda=trial.suggest_float("reg_lambda", 1e-4, 1e2, log=True),
    )
    xgb_opt = XGBRegressor(**xgb_params_opt)
    return score_dataset_XGB(X_train, y_train, xgb_opt)

study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=20)
xgb_params_opt = study.best_params

[I 2024-08-04 18:05:26,926] A new study created in memory with name: no-name-d7ce3c7f-aead-4e66-9f65-592709c120ec
[I 2024-08-04 18:05:35,510] Trial 0 finished with value: 27241.068706665126 and parameters: {'max_depth': 9, 'learning_rate': 0.03780655000387845, 'n_estimators': 1017, 'min_child_weight': 1, 'colsample_bytree': 0.37671819121850136, 'subsample': 0.4136233673745704, 'reg_alpha': 2.5106985831329425, 'reg_lambda': 0.012083649871387554}. Best is trial 0 with value: 27241.068706665126.
[I 2024-08-04 18:05:42,405] Trial 1 finished with value: 25802.823293376943 and parameters: {'max_depth': 3, 'learning_rate': 0.016761835856337916, 'n_estimators': 3356, 'min_child_weight': 1, 'colsample_bytree': 0.8570739066332274, 'subsample': 0.6446768356472575, 'reg_alpha': 0.3161289122042821, 'reg_lambda': 0.0009924467072650967}. Best is trial 1 with value: 25802.823293376943.
[I 2024-08-04 18:05:49,823] Trial 2 finished with value: 27788.794011676244 and parameters: {'max_depth': 10, 'learni

# Training the final model

In [26]:
xgb_final = XGBRegressor(**xgb_params_opt)
score_dataset_XGB(X_train, y_train, xgb_final)

25802.823293376943

In [27]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1460 entries, 1 to 1460
Data columns (total 74 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   MSSubClass        1460 non-null   int8   
 1   MSZoning          1460 non-null   int8   
 2   LotFrontage       1460 non-null   float64
 3   LotArea           1460 non-null   int64  
 4   Street            1460 non-null   int8   
 5   Alley             1460 non-null   int8   
 6   LotShape          1460 non-null   int8   
 7   LandContour       1460 non-null   int8   
 8   Utilities         1460 non-null   int8   
 9   LotConfig         1460 non-null   int8   
 10  LandSlope         1460 non-null   int8   
 11  Neighborhood      1460 non-null   int8   
 12  Condition1        1460 non-null   int8   
 13  Condition2        1460 non-null   int8   
 14  BldgType          1460 non-null   int8   
 15  HouseStyle        1460 non-null   int8   
 16  OverallQual       1460 non-null   int8   
 17  

In [30]:
pip install eli5

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.2/216.2 kB 2.6 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 kB 4.2 MB/s eta 0:00:00
  Created wheel for eli5: filename=eli5-0.13.0-py2.py3-none-any.whl size=107719 sha256=c5d83a1786a71d3710eaacf7ffb4552507238225d502533c7247e1d2ecea2430
  Stored in directory: /Users/anja/Library/Caches/pip/wheels/ec/68/a9/de7d374ecb6f53462ce0eec8326fbab91b6228c82e67428d0d
Successfully built eli5
Note: you may need to restart the kernel to use updated packages.


In [37]:
X_train, X_test = create_features(df_train, df_test)
y_train = df_train.loc[:, "SalePrice"]

final_model = XGBRegressor(**xgb_params_opt)
final_model.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.8570739066332274, device=None,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, feature_types=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=0.016761835856337916, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=3, max_leaves=None,
             min_child_weight=1, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=3356, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)